In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/sample.csv.csv
/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv
/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv


In [2]:
sample_data = pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/sample.csv.csv")
train_data = pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv")
test_data = pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv")

In [3]:
# print(sample_data.shape, "\n")
# print(sample_data.isnull().sum(),"\n")
# print(sample_data.info(),"\n")
# print(sample_data.head(),"\n")
# sample_data.describe()

In [4]:
# print(train_data.shape, "\n")
# print(train_data.isnull().sum(),"\n")
# print(train_data.info(),"\n")
# print(train_data.head(),"\n")
# train_data.describe()

In [5]:
# print(test_data.shape, "\n")
# print(test_data.isnull().sum(),"\n")
# print(test_data.info(),"\n")
# print(test_data.head(),"\n")
# test_data.describe()

In [6]:
# train_data.duplicated().value_counts()

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
#Data Visualisation
# sampled_data = train_data.sample(n=1000, random_state=42)
# sns.pairplot(sampled_data)
# plt.subplots_adjust(hspace = 0.5,wspace = 0.3)
# plt.show()

In [9]:
#Data Visualisation
# train_data.hist(bins = 100, figsize=(15, 10))
# plt.subplots_adjust(hspace = 0.5,wspace = 0.5)
# plt.show()

In [10]:
#Data Preprocessing
from datetime import datetime
import math

ride_duration=[]
ride_start=[]
rode_stop=[]
for i in range(len(train_data)):
    ts1 = datetime.strptime(train_data.iloc[i,1], "%Y-%m-%d %H:%M:%S")
    ts2 = datetime.strptime(train_data.iloc[i,2], "%Y-%m-%d %H:%M:%S")
    duration = abs(ts2 - ts1)
    ride_duration.append(duration.total_seconds())
    ride_start.append(ts1.strftime("%H"))
    rode_stop.append(ts2.strftime("%H"))
train_data["ride_duration"]=ride_duration
train_data["ride_start"]=ride_start
train_data["ride_stop"]=ride_start

ride_duration=[]
ride_start=[]
rode_stop=[]
for i in range(len(test_data)):
    ts1 = datetime.strptime(test_data.iloc[i,1], "%Y-%m-%d %H:%M:%S")
    ts2 = datetime.strptime(test_data.iloc[i,2], "%Y-%m-%d %H:%M:%S")
    duration = abs(ts2 - ts1)
    ride_duration.append(duration.total_seconds())
    ride_start.append(ts1.strftime("%H"))
    rode_stop.append(ts2.strftime("%H"))
test_data["ride_duration"]=ride_duration
test_data["ride_start"]=ride_start
test_data["ride_stop"]=ride_start

In [11]:
# print(train_data.shape, "\n")
# print(train_data.isnull().sum(),"\n")
# print(train_data.info(),"\n")
# print(train_data.head(),"\n")
# train_data.describe()

In [12]:
#Data Visualisation
# features = ['passenger_count','extra','tip_amount','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge','Airport_fee']
# time_feature = 'ride_duration'

# melted_data = train_data.melt(id_vars=[time_feature], value_vars=features, var_name='feature', value_name='value')

# plt.figure(figsize=(12, 6))
# sns.lineplot(data=melted_data, x=time_feature, y='value', hue='feature')

# plt.xlabel('Time or Continuous Variable')
# plt.ylabel('Feature Values')
# plt.title('Line Plot of 8 Features Over Time')
# plt.legend(title='Features', bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.show()

In [13]:
#Data encoding
new_train = train_data.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'], axis=1)
new_train = pd.get_dummies(new_train,columns=['store_and_fwd_flag','payment_type'], dtype='int')
# print(new_train.isnull().sum())
for column in new_train.columns:
    mode = new_train[column].mode()[0]
    new_train[column].fillna(mode, inplace=True)
# print(new_train.isnull().sum())

new_test = test_data.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'], axis=1)
encoded_test = pd.get_dummies(new_test,columns=['store_and_fwd_flag','payment_type'], dtype='int')
# print(encoded_test.isnull().sum())
for column in encoded_test.columns:
    mode = encoded_test[column].mode()[0]
    encoded_test[column].fillna(mode, inplace=True)
# print(encoded_test.isnull().sum())

X_train = new_train.drop(['total_amount'], axis=1)
y_train = new_train['total_amount']
# print(X_train.head())
# print(y_train.head())

In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

model = make_pipeline(PolynomialFeatures(2), LinearRegression())
model.fit(X_train, y_train)
val_predictions = model.predict(encoded_test)

In [15]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(val_predictions)+1)]
submission["total_amount"] = val_predictions
submission.to_csv('submission.csv',index=False)